In [2]:
from keras.layers import Input, Flatten, Dense
from keras.models import Model

Using TensorFlow backend.
C:\Users\uSer\AnacondaMy\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\uSer\AnacondaMy\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\uSer\AnacondaMy\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\uSer\AnacondaMy\lib\site-packages\tensorflow\python\framework\dtypes.py

In [3]:
import numpy as np

In [4]:
r_E = list(range(6758))
np.random.shuffle(r_E)
r_N = list(range(6646))
np.random.shuffle(r_N)

In [5]:
from matplotlib import image
import os

def load_images_from_folder(folder,r):
    images = []
    for i in r: #os.listdir(folder[:6758//4]): # part of # make random choice
        names = os.listdir(folder)
        img = image.imread(os.path.join(folder,names[i]))
        if img is not None:
            images.append(img)
    return images
folder1 = "D:\\DATA_IDAO\\track_1\\track_1_rep\\idao_dataset\\train\\ER"
folder2 = "D:\\DATA_IDAO\\track_1\\track_1_rep\\idao_dataset\\train\\NR"

In [6]:
(x_train, y_train), (x_test, y_test) = (0,0),(0,0)

In [7]:
import numpy as np
from keras.utils import to_categorical
from matplotlib import pyplot as plt

In [8]:
images_E = load_images_from_folder(folder1,r_E[:6758//8])
images_N = load_images_from_folder(folder2,r_N[:6646//8])

In [9]:
# concatenate
y_train = np.array([[0]]*(6758//8) + [[1]]*(6647//8)) # (0,1)=E; (1,0)=N    

In [10]:
E_N_list = images_E + images_N

In [11]:
y_train.shape

(1674, 1)

In [12]:
x_train = np.array(E_N_list)


In [13]:
NUM_CLASSES = 2
y_train = to_categorical(y_train, NUM_CLASSES)

In [14]:
x_train.shape
x_train = np.reshape(x_train,(1674, 576, 576, 1))
print(x_train.shape)

(1674, 576, 576, 1)


In [15]:
train_d = list(zip(x_train,y_train))
np.random.shuffle(train_d)
np.random.shuffle(train_d)
np.random.shuffle(train_d)
np.random.shuffle(train_d)
x_train,y_train = list(zip(*train_d))    
x_train = np.array(x_train)
y_train = np.array(y_train)

In [16]:
import numpy as np
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras.layers import Conv2D, BatchNormalization, Dropout, LeakyReLU, Activation
from keras.optimizers import Adam

In [17]:
y_train.shape

(1674, 2)

In [18]:
x_train = x_train.astype('float32')

In [19]:
input_layer = Input((576,576,1))

x = Conv2D(filters = 10, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x) # 1
'''
x = Conv2D(filters = 10, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x) # 2


x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x) # 3

x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x) # 4
'''

x = Flatten()(x) 
x = Dense(128)(x) # layer of perceptrons

x = BatchNormalization()(x) # пакетная нормализация чтобы веса не взлетели

x = LeakyReLU()(x) # в качестве активации

x = Dropout(rate = 0.5)(x) # как я понял слой прореживания

x = Dense(NUM_CLASSES)(x)

output_layer = Activation('softmax')(x) # в качестве активации на последнем слое

model = Model(input_layer, output_layer)

In [20]:
opt = Adam(lr=0.005)

In [23]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [22]:
model.train_on_batch(x_train, y_train)
# generator

KeyboardInterrupt: 

In [37]:
images_E_test = load_images_from_folder(folder1,r_E[(6758//16)*2:(6758//16)*3])
images_N_test = load_images_from_folder(folder2,r_N[(6646//16)*2:(6646//16)*3])

In [38]:
y_test = np.array([[0]]*(6758//16) + [[1]]*(6647//16)) # (0,1)=E; (1,0)=N  

In [39]:
x_test = np.array(images_E_test + images_N_test)

In [40]:
y_test = to_categorical(y_test, NUM_CLASSES)

In [41]:
test_d = list(zip(x_test,y_test))
np.random.shuffle(test_d)
np.random.shuffle(test_d)
np.random.shuffle(test_d)
np.random.shuffle(test_d)
x_test,y_test = list(zip(*test_d))    
x_test = np.array(x_test)
y_test = np.array(y_test)

In [42]:
x_test.shape

(837, 576, 576)

In [43]:
print(model.evaluate(x_test, y_test, batch_size = 1674))

837/837 [==============================] - 22s 27ms/step
[0.6933548450469971, 0.5041816234588623]


In [66]:
'''
input_layer = Input((576,576,1))
x = Conv2D(filters = 32, kernel_size = 3
, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x) # 1

x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x) # 2

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x) # 3

x = Conv2D(filters = 64, kernel_size = 3, strides = 2, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x) # 4

x = Flatten()(x) 
x = Dense(128)(x) # layer of perceptrons

x = BatchNormalization()(x) # пакетная нормализация чтобы веса не взлетели

x = LeakyReLU()(x) # в качестве активации

x = Dropout(rate = 0.5)(x) # как я понял слой прореживания

x = Dense(NUM_CLASSES)(x)

output_layer = Activation('softmax')(x) # в качестве активации на последнем слое

model = Model(input_layer, output_layer)
'''